# Image Classification Project 6
Choose three classes from the Open Images Dataset. Train a neural net that is able to classify images into these three categories.



In [11]:
classes = ["Cat", "Dog", "Person"]

## Dataset
https://storage.googleapis.com/openimages/web/visualizer/index.html?type=detection

## Base model
VGG 19

In [12]:
# imports
import tensorflow as tf
from keras.applications import VGG19
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import fiftyone as fo
import fiftyone.zoo as foz

print(foz.list_zoo_datasets())

['activitynet-100', 'activitynet-200', 'bdd100k', 'caltech101', 'caltech256', 'cifar10', 'cifar100', 'cityscapes', 'coco-2014', 'coco-2017', 'fashion-mnist', 'fiw', 'hmdb51', 'imagenet-2012', 'imagenet-sample', 'kinetics-400', 'kinetics-600', 'kinetics-700', 'kinetics-700-2020', 'kitti', 'kitti-multiview', 'lfw', 'mnist', 'open-images-v6', 'open-images-v7', 'quickstart', 'quickstart-geo', 'quickstart-groups', 'quickstart-video', 'sama-coco', 'ucf101', 'voc-2007', 'voc-2012']


In [13]:
# Path to the directory where the images are stored
base_dir = './dataset'
n_images = 100  # number of images per class

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    max_samples=n_images,
    seed=51,
    shuffle=True,
    label_types=["classifications"],
    classes= classes,
    dataset_name="cdp-dataset",
)




Necessary images already downloaded
Existing download of split 'train' is sufficient
Necessary images already downloaded
Existing download of split 'test' is sufficient
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'cdp-dataset'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [14]:
print(dataset.info.keys())

session = fo.launch_app(dataset.view())

dict_keys(['hierarchy', 'classes_map'])


## Task
1. Preparation: Split dataset into a 70/30 Train/test split


In [16]:
# Define parameters for the loader
batch_size = 20
img_height = 224
img_width = 224

# Load the training data
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)  # set validation split

train_generator = train_datagen.flow_from_directory(
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # set as training data

# Load the validation data
validation_generator = train_datagen.flow_from_directory(
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # set as validation data

TypeError: flow_from_directory() missing 1 required positional argument: 'directory'

2. Train a VGG19 network from scratch (randomly initialized weights) and estimate the testset accuracy.

In [9]:
# load a vgg19 with random init weights
random_base_vgg19 = VGG19(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# TODO estimate testset accuracy



3. Transfer learning: Use an imagenet pretrained VGG19 network, train the model and estimate the testset accuracy. Show the differences in loss and accuracy of the plain and pre trained network over the first 10 epochs.

In [10]:
# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10)

# Save the model
model.save('models/model.h5')


Epoch 1/10
157/157 [==============================] - 447s 3s/step - loss: 1.8531 - accuracy: 0.5722 - val_loss: 0.8886 - val_accuracy: 0.6269
Epoch 2/10
157/157 [==============================] - 482s 3s/step - loss: 0.7020 - accuracy: 0.7166 - val_loss: 0.7868 - val_accuracy: 0.6649
Epoch 3/10
157/157 [==============================] - 390s 2s/step - loss: 0.6338 - accuracy: 0.7422 - val_loss: 0.7973 - val_accuracy: 0.6552
Epoch 4/10
157/157 [==============================] - 472s 3s/step - loss: 0.5546 - accuracy: 0.7604 - val_loss: 0.8229 - val_accuracy: 0.6709
Epoch 5/10
157/157 [==============================] - 436s 3s/step - loss: 0.5016 - accuracy: 0.7885 - val_loss: 0.7594 - val_accuracy: 0.6896
Epoch 6/10
157/157 [==============================] - 449s 3s/step - loss: 0.4661 - accuracy: 0.8038 - val_loss: 0.7592 - val_accuracy: 0.6881
Epoch 7/10
157/157 [==============================] - 485s 3s/step - loss: 0.4230 - accuracy: 0.8329 - val_loss: 0.8835 - val_accuracy: 0.6604

4. Data cleansing: Remove “bad” images from the dataset. Which did you remove? How many? Discuss results.

5. Add data augmentation and train again, discuss results

6. Rebuild VGG19. After layer block4_conv4 (25, 25, 512):
    - Random flip
    - Random contrast
    - Random translation

7. Test a few of your own images and present the results
    - Add inception layer with dimensionality reduction (no of output filters should be 512, choose own values for the filter dimensionality reduction in 1x1 layers)
    - Add conv layer (kernel 1x1,  filters 1024, padding valid, stride 1, activation leaky relu)
    - Add conv layer (kernel 3x3,  filters 1024, padding same, stride 1, activation relu)
    - Freeze conv2 layers and before

8. Answer the following questions:
    - What accuracy can be achieved? What is the accuracy of the train vs. test set?
    - On what infrastructure did you train it? What is the inference time?
    - What are the number of parameters of the model?
    - Which categories are most likely to be confused by the algorithm? Show results in a confusion matrix.

Compare the results of the experiments.